In [13]:
import jsonlines
import spacy
import spacy.language
from spacy.tokens import Doc
from spacy.scorer import Scorer
from spacy.vocab import Vocab
import statsmodels
import pandas as pd

In [3]:
# path to jsonl overlap files
path_coco = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_coco.jsonl"
path_graf = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_leo.jsonl"
path_hoff = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_jona.jsonl"
path_jthn = r"/Users/jhoff/Universität St.Gallen/STUD-Capstoneproject Tell 6 - Dokumente/General/02-Coding/01-Data/20_overlap/overlap_total/overlap_jonathan.jsonl"

In [11]:
def jsonl_to_list(path):
    """takes path to jsonl file and returns list of dicts"""
    
    with jsonlines.open(path) as reader:
        list_of_dicts = list(reader)

    return list_of_dicts

In [47]:
def label_per_token(recipe):
    """takes annotated recipes as input and returns a dict that maps label to every token"""

    amount_tokens = len(recipe["tokens"])

    all_token_dict = { tuple(range(token["start"], token["end"]+1)) : "None" for token in recipe["tokens"]}    # each token as list and value "None"

    for span in recipe["spans"]:

        all_chars = list(range(span["start"], span["end"]+1))

        label = span["label"]

        for char in all_chars:              #compare all characters with all token characters
            for tok in all_token_dict.keys():
                if char in tok: 
                    all_token_dict[tok] = label
    
    print(f"Amount tokens: {amount_tokens} vs. Length dict: {len(all_token_dict.keys())}")

    return all_token_dict #{token: label}

In [ ]:
def token_table_per_recipe(recipe):
    """takes example annotated recipe and creates empty dict with token_start_char as indices and ent classes as column labels"""
    
    index = [tuple(range(start, end+1)) for token in recipe["tokens"]]

    columns = ["Z", "TOOL", "VERB", "ATTR", "PRÄP", "ZEITP", "DAUER", "TEMP"]

    token_table = pd.DataFrame(index=index, columns=columns)

    return token_table  #pd style table

In [9]:
def calculate_kappa(table):
    """takes table with tokens and labels as input and returns kappa"""

    kappa = statsmodels.stats.inter_rater.fleiss_kappa(table, method="fleiss")


    return kappa

In [14]:
# list of individual overlap dicts
ov_recipes_coco = jsonl_to_list(path_coco)
ov_recipes_graf = jsonl_to_list(path_graf)
ov_recipes_hoff = jsonl_to_list(path_hoff)
ov_recipes_jthn = jsonl_to_list(path_jthn)


In [46]:
x = ov_recipes_coco[0]
print(x["text"])

Für die Barbecue Sauce das Öl erhitzen, Zwiebeln glasig anbraten. Knoblauch, Nelken, Tabasco und Kümmel zugeben und 2 Minuten brutzeln lassen. Alle anderen Zutaten (außer den Rippchen) zugeben, aufkochen lassen und 20 Minuten köcheln lassen, bis die Sauce eindickt. Vom Herd nehmen und alles durch ein Sieb passieren.Bei normalen, dünnen Rippchen, diese kurz auf dem Grill anbraten und dann großzügig nach jedem wenden mit der Sauce einpinseln. Die entstehende Kruste darf ruhig etwas schwarz werden, schmeckt extrem lecker. Wichtig ist es, die Rippchen in Bewegung zu halten und im Minutentakt zu wenden und einzupinseln!Dicke Rippchen vor dem Einpinseln länger vorbraten. Oder vorher in heißem Wasser, nicht kochendem, 30 Minuten bis eine Stunde ziehen lassen.Dazu passt ein guter schwäbischer Kartoffelsalat und jede Menge kaltes Bier.


In [32]:
x = tuple(range(0, 5))
print(type(x))

<class 'tuple'>


In [48]:
dict = label_per_token(ov_recipes_coco[0])
print(dict)

Amount tokens: 150 vs. Length dict: 150
{(0, 1, 2, 3): 'None', (4, 5, 6, 7): 'None', (8, 9, 10, 11, 12, 13, 14, 15, 16): 'None', (17, 18, 19, 20, 21, 22): 'None', (23, 24, 25, 26): 'None', (27, 28, 29): 'Z', (30, 31, 32, 33, 34, 35, 36, 37, 38): 'V', (38, 39): 'V', (40, 41, 42, 43, 44, 45, 46, 47, 48): 'Z', (49, 50, 51, 52, 53, 54, 55): 'ATTR', (56, 57, 58, 59, 60, 61, 62, 63, 64): 'V', (64, 65): 'V', (66, 67, 68, 69, 70, 71, 72, 73, 74, 75): 'Z', (75, 76): 'Z', (77, 78, 79, 80, 81, 82, 83): 'Z', (83, 84): 'Z', (85, 86, 87, 88, 89, 90, 91, 92): 'Z', (93, 94, 95, 96): 'None', (97, 98, 99, 100, 101, 102, 103): 'Z', (104, 105, 106, 107, 108, 109, 110, 111): 'V', (112, 113, 114, 115): 'None', (116, 117): 'DAUER', (118, 119, 120, 121, 122, 123, 124, 125): 'DAUER', (126, 127, 128, 129, 130, 131, 132, 133, 134): 'V', (135, 136, 137, 138, 139, 140, 141): 'None', (141, 142): 'None', (143, 144, 145, 146, 147): 'Z', (148, 149, 150, 151, 152, 153, 154, 155): 'Z', (156, 157, 158, 159, 160, 161, 162